<a href="https://colab.research.google.com/github/DavorJordacevic/OpenCV/blob/master/Depth_on_Car_Stereo_System_From_Image_Sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preprocessing

In [ ]:
!wget "https://liquidtelecom.dl.sourceforge.net/project/mrpt/Datasets%20%28Rawlogs%29/Datasets/Karlsruhe_sequences/2010_03_09_drive_0023.zip"

--2019-08-25 10:25:04--  https://liquidtelecom.dl.sourceforge.net/project/mrpt/Datasets%20%28Rawlogs%29/Datasets/Karlsruhe_sequences/2010_03_09_drive_0023.zip
Resolving liquidtelecom.dl.sourceforge.net (liquidtelecom.dl.sourceforge.net)... 197.155.77.8
Connecting to liquidtelecom.dl.sourceforge.net (liquidtelecom.dl.sourceforge.net)|197.155.77.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61304933 (58M) [application/octet-stream]
Saving to: ‘2010_03_09_drive_0023.zip’

2010_03_09_drive_00 100%[===================>]  58.46M  12.3MB/s    in 6.0s    

2019-08-25 10:25:11 (9.74 MB/s) - ‘2010_03_09_drive_0023.zip’ saved [61304933/61304933]



In [ ]:
import zipfile

with zipfile.ZipFile("2010_03_09_drive_0023.zip","r") as zip_ref:
    zip_ref.extractall("")



---



---



---



---



---



---



In [ ]:
import cv2
import glob
import numpy as np

In [ ]:
filenameLeft = glob.glob("2010_03_09_drive_0023/2010_03_09_drive_0023_Images/I1*.png")
filenameRight = glob.glob("2010_03_09_drive_0023/2010_03_09_drive_0023_Images/I2*.png")

filenameLeft.sort()
filenameRight.sort()

filenameLeft = np.asarray(filenameLeft)
filenameRight = np.asarray(filenameRight)

In [ ]:
images_left = [cv2.imread(file) for file in filenameLeft]
images_right = [cv2.imread(file) for file in filenameRight]

images_left = np.asarray(images_left)
images_right = np.asarray(images_right)

In [ ]:
video = list()

for i in range (images_left.shape[0]):
  image_left = images_left[i]
  image_right= images_right[i]

  window_size = 3
  min_disp = 0
  num_disp = 160-min_disp

  left_matcher = cv2.StereoSGBM_create(
      minDisparity=0,
      numDisparities=160,             
      blockSize=5,
      P1=8 * 3 * window_size ** 2,
      P2=32 * 3 * window_size ** 2,
      disp12MaxDiff=1,
      uniquenessRatio=15,
      speckleWindowSize=0,
      speckleRange=2,
      preFilterCap=63,
      mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
  )


  right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)
  
  # Disparity map filter based on Weighted Least Squares filter

  # The amount of regularization during filtering. Larger values force filtered 
  # disparity map edges to adhere more to source image edges.
  # Typical value is 8000
  wls_filter_lambda = 80000 #lambda is reserved word
  wls_filter_sigma = 1.3 # how sensitive the filtering process is to source image edges

  # Create a filter 
  wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
  wls_filter.setLambda(wls_filter_lambda)
  wls_filter.setSigmaColor(wls_filter_sigma)


  left_disparity = left_matcher.compute(image_left, image_right)
  left_disparity = np.int16(left_disparity)

  right_disparity = right_matcher.compute(image_right, image_left)
  right_disparity = np.int16(right_disparity)

  disparity = wls_filter.filter(left_disparity, image_left, None, right_disparity)

  disparity = cv2.normalize(src=disparity, dst=disparity, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX);
  disparity = np.uint8(disparity)
  
  disparity = cv2.applyColorMap(disparity, cv2.COLORMAP_JET)
  
  video.append(disparity)

In [ ]:
video = np.asarray(video)

In [ ]:
# Define the codec and create VideoWriter object.The output is stored in 'output.avi' file.
# Define the fps to be equal to 10. Also frame size is passed.

videowriter = cv2.VideoWriter('disparity_py.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (video[0].shape[1],video[0].shape[0]))

for i in range (video.shape[0]):
  videowriter.write(video[i])
  
videowriter.release()